In [1]:
from random import random
import numpy as np
import math

In [2]:
def model():
    x = 0
    while True:
        sv = random()
        x += 1
        if sv < p:
            break
    return x


def get_q(interval, y_lst, p):
    q = 0
    for i in range(len(y_lst)):
        if y_lst[i] >= interval[0] and y_lst[i] < interval[1]:
            q += p[i]
    return q

def get_nj(interval, y_lst, y_values):
    nj = 0
    for i in range(len(y_lst)):
        if y_lst[i] >= interval[0] and y_lst[i] < interval[1]:
            nj += y_values[i]
    return nj

def get_y_dict(n):
    y_dict = dict()

    for i in range(n):
        x = model()
        if x not in y_dict.keys():
            y_dict[x] = 1
        else:
            y_dict[x] += 1
    return y_dict

def get_y(n):
    y_dict = get_y_dict(n)
    y_lst = sorted(y_dict.keys())
    y_values = [y_dict[y_lst[i]] for i in range(len(y_lst))]
    return y_dict, y_lst, y_values

In [9]:
p = float(input("\nВведите вероятность\n"))
n = int(input("\nВведите количество экспериментов\n"))
y_dict, y_lst, y_values = get_y(n)


Введите вероятность
0.3

Введите количество экспериментов
10000


In [10]:
k = int(input("Введите количество точек: "))
z = [float("-inf")]
for i in range(k):
    z.append(float(input(f"Введите z{i+1} = ")))
z.append(float("inf"))
print("Интервалы разбиения: ", z)

Введите количество точек: 9
Введите z1 = 1
Введите z2 = 4
Введите z3 = 6
Введите z4 = 8
Введите z5 = 9
Введите z6 = 10
Введите z7 = 23
Введите z8 = 4
Введите z9 = 1
Интервалы разбиения:  [-inf, 1.0, 4.0, 6.0, 8.0, 9.0, 10.0, 23.0, 4.0, 1.0, inf]


In [11]:
def get_R0(y_lst, y_values, z, n, p):
    P_lst = []
    for i in range(len(y_lst)):
        P = p * ((1 - p) ** (y_lst[i] - 1))
        P_lst.append(P)
    q = []
    n_lst = []
    for i in range(len(z)-1):
        q.append(get_q((z[i], z[i+1]), y_lst, P_lst))
        n_lst.append(get_nj((z[i], z[i+1]), y_lst, y_values))
    R0 = 0
    for j in range(len(q)):
        try:
            R0 += ((n_lst[j] - n*q[j]) ** 2) / (n*q[j])
        except ZeroDivisionError:
            pass
    return R0


R0_lst = []
for i in range(100):
    y_dict, y_lst, y_values = get_y(n)
    R0_lst.append(get_R0(y_lst, y_values, z, n, p))

    
def hypothesis(p):
    alpha = float(input("Введите уровень значимости α = "))
    def get_integr(a, b):
        r = k
        def func(z):
            try:
                return (z**(0.5*r-1)).real * math.exp(-0.5*z) * (math.gamma(0.5*r) ** (-1)) * (2 ** (-0.5*r))
            except ZeroDivisionError:
                return 0
        razb = 1000
        res = sum([(func(a+(b-a)*((k-1)/razb)) + func(a+(b-a)*(k/razb)))*((b-a)/(2*razb)) for k in range(razb)])
        return res
    f = [get_integr(0, el) for el in R0_lst]
    reject = [0 for j in f if j.real >= 1 - alpha]
    accept = [1 for j in f if j.real < 1 - alpha]

    print(f"Количество отвергнутых гипотез Н0 о геометрическом распределении = {len(reject)}")
    print(f"Количество принятых гипотез Н0 о геометрическом распределении = {len(accept)}\n")

for i in range(3):
    hypothesis(p)

Введите уровень значимости α = 0.05
Количество отвергнутых гипотез Н0 о геометрическом распределении = 1
Количество принятых гипотез Н0 о геометрическом распределении = 99

Введите уровень значимости α = 0.01
Количество отвергнутых гипотез Н0 о геометрическом распределении = 0
Количество принятых гипотез Н0 о геометрическом распределении = 100

Введите уровень значимости α = 0.55
Количество отвергнутых гипотез Н0 о геометрическом распределении = 11
Количество принятых гипотез Н0 о геометрическом распределении = 89

